In [1]:
import pandas as pd
import datetime

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [2]:
input_price = pd.read_excel("daily_trading_info_train.xlsx", sheet_name = None)

In [3]:
stock_index = []
stock_index = list(input_price.keys())[0:]

In [4]:
results = pd.read_excel("strategy_results_train.xlsx", sheet_name = "strategy_results_train")

In [5]:
import copy

In [6]:
dataset = copy.deepcopy(results)

##### START: Caculating the five-day average close price of SHSE.600085 at 2015-01-31

In [7]:
def ma(input_price, trade_company, trade_date_datetime, ma_length):
    df = input_price[trade_company]
    date_index = 0
    for i in range(0, df.shape[0]):
        if(df.loc[i][1] >= trade_date_datetime):
#             print(i, df.iloc[i][1])
            date_index = i
            break
    ma = 0
    ma_length = 5
    for reverse in range(0, ma_length):
        ma = ma + df.iloc[date_index-reverse][5]
    ma = ma/ ma_length
    return ma

In [8]:
temp = []
for i in range(0, dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = dataset.iloc[i][0].split("_")[0]
    trade_date    = dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma(input_price, trade_company, trade_date_datetime, ma_length = 5))
#     print(i, trade_company, trade_date, trade_date_datetime)

0 2020-05-18 15:47:54.546590
1000 2020-05-18 15:49:48.740233
2000 2020-05-18 15:51:38.914569
3000 2020-05-18 15:53:31.990190
4000 2020-05-18 15:55:25.862786
5000 2020-05-18 15:57:13.862729
6000 2020-05-18 15:59:11.327598
7000 2020-05-18 16:00:58.443150
8000 2020-05-18 16:02:51.254727
9000 2020-05-18 16:04:44.777044
10000 2020-05-18 16:06:30.507336
11000 2020-05-18 16:08:12.824794
12000 2020-05-18 16:10:08.496567
13000 2020-05-18 16:12:04.104490
14000 2020-05-18 16:14:03.432399
15000 2020-05-18 16:16:00.129372
16000 2020-05-18 16:17:58.030345
17000 2020-05-18 16:19:54.840908
18000 2020-05-18 16:21:42.991056
19000 2020-05-18 16:23:39.477169
20000 2020-05-18 16:25:28.385955
21000 2020-05-18 16:27:22.457250
22000 2020-05-18 16:29:11.838519
23000 2020-05-18 16:31:01.798866
24000 2020-05-18 16:32:56.323571
25000 2020-05-18 16:34:42.410395
26000 2020-05-18 16:36:34.087643
27000 2020-05-18 16:38:16.100777
28000 2020-05-18 16:40:13.046081
29000 2020-05-18 16:42:04.619300
30000 2020-05-18 16:43:

In [9]:
pd.DataFrame(temp, columns=["ma"]).head()
temp = pd.DataFrame(temp, columns=["ma"])

In [10]:
temp

,ma
0,21.548000
1,22.910000
2,22.492000
3,26.336000
4,32.718000
...,...
31942,58.034000
31943,55.264000
31944,55.188000
31945,46.204000


In [11]:
dataset.shape

(31947, 2)

In [12]:
result = pd.concat([dataset, pd.DataFrame(temp).reindex(dataset.index)], axis=1)
result.head()

,id,label,ma
0,SHSE.600085_201501,0,21.548
1,SHSE.600085_201502,1,22.910
2,SHSE.600085_201503,1,22.492
3,SHSE.600085_201504,1,26.336
4,SHSE.600085_201505,1,32.718


In [13]:
result.to_excel("train_output.xlsx")  

In [14]:
y = pd.DataFrame(result["label"])
X = pd.DataFrame(result["ma"])

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [25]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
print("train f1: ", f1_score(y_train, clf.predict(X_train)))
print("valid f1: ", f1_score(y_valid, clf.predict(X_valid)))
print("train accuracy: ", accuracy_score(y_train, clf.predict(X_train)))
print("valid accuracy: ", accuracy_score(y_valid, clf.predict(X_valid)))
print("train recall_score: ", recall_score(y_train, clf.predict(X_train)))
print("valid recall_score: ", recall_score(y_valid, clf.predict(X_valid)))
print("train precision_score: ", precision_score(y_train, clf.predict(X_train)))
print("valid precision_score: ", precision_score(y_valid, clf.predict(X_valid)))
fpr, tpr, thresholds = metrics.roc_curve(y_train.to_numpy().flatten(), clf.predict(X_train).flatten(), pos_label=1)
print("train auc: ", metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_valid.to_numpy().flatten(), clf.predict(X_valid).flatten(), pos_label=1)
print("valid auc: ", metrics.auc(fpr, tpr))

train f1:  0.06548921537980619
valid f1:  0.07668711656441718
train accuracy:  0.5321047071252495
valid accuracy:  0.5289514866979655
train recall_score:  0.03504809703053116
valid recall_score:  0.04101049868766404
train precision_score:  0.4982164090368609
valid precision_score:  0.589622641509434
train auc:  0.5020116238718307
valid auc:  0.507489091354604


## apply all the train data for model fitting

In [27]:
y_train = pd.DataFrame(result["label"])
X_train = pd.DataFrame(result["ma"])
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### 開始處理 test set

In [18]:
test_input_price = pd.read_excel("daily_trading_info_test.xlsx", sheet_name = None)

In [19]:
test_stock_index = []
test_stock_index = list(test_input_price.keys())[0:]

In [34]:
test_results = pd.read_csv("sampleSubmission.csv")

In [35]:
test_results.head()

,id,label
0,SHSE.600085_201901,0
1,SHSE.600085_201902,0
2,SHSE.600085_201903,0
3,SHSE.600085_201904,0
4,SHSE.600085_201905,0


In [36]:
import copy

In [40]:
test_dataset = copy.deepcopy(test_results)

In [ ]:
temp = []
for i in range(0, test_dataset.shape[0]):
    if(i%1000==0):
        print(i, datetime.datetime.now())
    trade_company = test_dataset.iloc[i][0].split("_")[0]
    trade_date    = test_dataset.iloc[i][0].split("_")[1]
    trade_date_datetime = datetime.date(int(trade_date[:4]), int(trade_date[4:]), 1) - datetime.timedelta(days=1)
    temp.append(ma(test_input_price, trade_company, trade_date_datetime, ma_length = 5))
#     print(i, trade_company, trade_date)

0 2020-05-18 17:09:20.662834
1000 2020-05-18 17:09:44.955338
2000 2020-05-18 17:10:08.896516
3000 2020-05-18 17:10:33.239890
4000 2020-05-18 17:10:57.208650
5000 2020-05-18 17:11:21.492402
6000 2020-05-18 17:11:45.357077
7000 2020-05-18 17:12:09.865811
8000 2020-05-18 17:12:33.811249


In [38]:
len(temp)

11363

In [39]:
test_dataset.shape[0]

11363

In [41]:
temp = pd.DataFrame(temp, columns=["ma"])
test_result = pd.concat([test_dataset, temp], axis=1)
test_result.head()

,id,label,ma
0,SHSE.600085_201901,0,25.552
1,SHSE.600085_201902,0,26.562
2,SHSE.600085_201903,0,29.076
3,SHSE.600085_201904,0,29.810
4,SHSE.600085_201905,0,31.326


In [42]:
test_result.to_excel("test_output.xlsx")  

In [43]:
test_X = pd.DataFrame(test_result["ma"])

In [44]:
test_X.head(10)

,ma
0,25.552
1,26.562
2,29.076
3,29.810
4,31.326
5,29.536
6,29.036
7,28.268
8,28.168
9,27.152


In [45]:
pd.DataFrame(clf.predict(test_X)).to_excel("test_output.xlsx")  

##### END: Caculating the five-day average close price of SHSE.600085 at 2015-01-31